# Current GFW Hadoop process

This notebook will examine sample inputs and outputs of our current hadoop process. It will focus on calculating the area of tree cover loss within the footprint of a [single 10 degree by 10 degree tile](http://geojson.io/#data=data:application/json,%7B%22bbox%22%3A%5B-60%2C-10%2C-50%2C0%5D%2C%22features%22%3A%5B%7B%22bbox%22%3A%5B-60%2C-10%2C-50%2C0%5D%2C%22geometry%22%3A%7B%22coordinates%22%3A%5B%5B%5B-60%2C-10%5D%2C%5B-50%2C-10%5D%2C%5B-50%2C0%5D%2C%5B-60%2C0%5D%2C%5B-60%2C-10%5D%5D%5D%2C%22type%22%3A%22Polygon%22%7D%2C%22properties%22%3A%7B%22filename%22%3A%22loss.tif%22%2C%22id%22%3A%220%22%2C%22title%22%3A%22loss.tif%22%7D%2C%22type%22%3A%22Feature%22%7D%5D%2C%22type%22%3A%22FeatureCollection%22%7D).

Results are summarized by polyname (intact forest landscapes, world database of protected areas (WDPA), etc) and by the iso/adm1/adm2 codes. For our analysis, we'll focus only on the iso/adm1/adm2 areas contained entirely within our 10x10 area. These boundaries are in geometry/gadm36_aoi.geojson.

### Raster inputs

Raster inputs are from the [UMD Global Forest Change dataset](https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.5.html). Our loss raster is available [here](https://storage.googleapis.com/earthenginepartners-hansen/GFC-2017-v1.5/Hansen_GFC-2017-v1.5_lossyear_00N_060W.tif). It has values of 2001 - 2017 if tree cover loss occurred in any of those years. Our tree cover canopy density raster is available [here](https://storage.googleapis.com/earthenginepartners-hansen/GFC-2017-v1.5/Hansen_GFC-2017-v1.5_treecover2000_00N_060W.tif), and represents the tree cover canopy density in the year 2000).

Our process "joins" these two rasters, so that we're reporting the total area for each combination of tree cover loss and tree cover canopy density. This is currently stored in vector format as a TSV, but from a raster standpoint, this might be handled by stacking loss and tree cover density rasters as separate bands. 

### Polygon inputs

Polygons are input coverages for various areas of interest, intersected with our GADM admin boundary dataset. Example geometries for this analysis are in the `geometry/` folder. In addition to iso/adm1/adm2 information, each has `bound` fields for any dataset specific information we'd like to group by. Datasets in this example don't have any of these values, but a good example is our plantations data which includes the species and the age of each plantation.

### Output

Once our raster and polygon layers are intersected, we group and sum the output table to derive our summary statistics.

In [1]:
# OK let's look at the output!
import pandas as pd

df = pd.read_csv('loss_totals.csv')
df.head()

,polyname,bound1,bound2,bound3,bound4,iso,adm1,adm2,thresh,loss_year,area
0,admin,1,1,1,1,BRA,12,9,0,16,5.003145e+06
1,ifl_2013__landmark,1,1,1,1,BRA,12,9,75,11,1.028354e+06
2,ifl_2013__landmark,1,1,1,1,BRA,12,9,30,3,7.611101e+02
3,admin,1,1,1,1,BRA,12,9,10,15,1.183998e+05
4,wdpa,1,1,1,1,BRA,12,9,30,11,1.520655e+03


#### The thresh column
Most of the column names are self explanatory, i think, with the exception of thresh. thresh is the value for tree cover canopy density percent. While this value in the source raster ranges from 0 - 100, we group these values into different tree cover canopy density thresholds.
This scala code performs that lookup:
https://github.com/wri/spark-pip/blob/master/src/main/scala/com/esri/HansenUtils.scala#L5-L16


#### Final destination
We'll do some postprocessing on this data- joining it to similar tables (polyname / bounds / iso / adm1 / adm2) of tree cover density, gain, and geographic area. [Here's an example](https://production-api.globalforestwatch.org/v1/query/a20e9c0e-8d7d-422f-90f5-3b9bca355aaf?sql=SELECT%20*%20FROM%20data%20LIMIT%2010) of that dataset.

These stats are then pulled into our [country dashboard pages](https://www.globalforestwatch.org/dashboards/country/BRA), letting users investigate tree cover loss by year and tree cover threshold within various polygons.

Let's compare our results in this table to what we're displaying in the dashboards for Brazil state #12, county #9. (This county is contained entirely within our 10 x 10 degree tile, meaning our numbers from this example should line up exactly).

In [17]:
from IPython.display import IFrame
url = "https://www.globalforestwatch.org/embed/dashboards/country/BRA/12/9?widget=treeLoss&treeLoss=eyJ0aHJlc2hvbGQiOjc1fQ=="
IFrame(url, width=670, height=490)

In [2]:
# and now let's filter our dataframe to match
df[(df.polyname == 'admin') & 
   (df.adm1 == 12) & 
   (df.adm2 == 9) & 
   (df.thresh == 75)].sort_values('loss_year').head()

,polyname,bound1,bound2,bound3,bound4,iso,adm1,adm2,thresh,loss_year,area
291,admin,1,1,1,1,BRA,12,9,75,1,7.958408e+07
128,admin,1,1,1,1,BRA,12,9,75,2,2.291816e+08
435,admin,1,1,1,1,BRA,12,9,75,3,1.940724e+08
363,admin,1,1,1,1,BRA,12,9,75,4,2.044573e+08
385,admin,1,1,1,1,BRA,12,9,75,5,1.273664e+08


Looks great! Yearly loss numbers match up exactly. It's important to note that our values for  tree cover thresh other than 75 won't match what's displayed on the GFW dashboards. Threshold areas are cumulative, so the loss area for threshold 50 would actually be the data we have at 50, plus the data we have at 75. We handle this postprocessing in pandas after the hadoop output runs.